<a href="https://colab.research.google.com/github/jaalvalcan/GEE_index_sets/blob/main/GIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee



In [ ]:

ee.Authenticate()
ee.Initialize(project='ee-jaalvalcan')

In [ ]:
# 2. Define Chinguetti ROI (Lon/Lat order)
roi = ee.Geometry.Polygon([[
    [-12.377, 20.450],
    [-12.377, 20.470],
    [-12.350, 20.470],
    [-12.350, 20.450],
    [-12.377, 20.450]
]])

In [ ]:
# 3. Processing Function
def process_year(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)

    # Get Sentinel-2 Median Composite
    img = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
           .filterBounds(roi)
           .filterDate(start, end)
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
           .median()
           .clip(roi))

    # Calculate NDVI
    # Formula: (NIR - Red) / (NIR + Red) -> (B8 - B4) / (B8 + B4)
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Create Visualizations
    true_color = img.visualize(bands=['B4', 'B3', 'B2'], min=0, max=4000)
    ndvi_color = ndvi.visualize(min=0, max=0.5, palette=['red', 'yellow', 'green'])

    return ee.Image.cat([true_color.rename(['R', 'G', 'B']),
                         ndvi_color.rename(['nR', 'nG', 'nB'])]).set('year', year)

# 4. Create the Collections
years = ee.List.sequence(2018, 2025)
processed_collection = ee.ImageCollection.fromImages(years.map(process_year))

# 5. Generate True Color GIF URL
true_color_url = processed_collection.select(['R', 'G', 'B']).getVideoThumbURL({
    'dimensions': 600,
    'region': roi,
    'framesPerSecond': 1,
    'format': 'gif'
})

# 6. Generate NDVI GIF URL
ndvi_url = processed_collection.select(['nR', 'nG', 'nB']).getVideoThumbURL({
    'dimensions': 600,
    'region': roi,
    'framesPerSecond': 4,
    'format': 'gif'
})

print(f"1. True Color GIF: {true_color_url}")
print(f"2. NDVI GIF (Green = Vegetation, Red = Sand/Rock): {ndvi_url}")